# 目標和進展
在這個筆記本中，我們嘗試重新創建 [G-Research 加密競賽教程](https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition/notebook#) 中描述的目標計算建立你的預測模型）。

此處提供的代碼側重於正確的目標計算，並未進行優化。當前的筆記本版本提供了接近的結果，但並不理想。平均而言，差異很小，但對於某些行給出的值超出了正常目標範圍。我希望我們能在主持人、kaggle 團隊和社區的幫助下解決這個問題。

**從 2021 年 11 月 9 日起更新**

感謝 Ernesto Budia 的調查（參見他的 [notebook](https://www.kaggle.com/ebudia/recreating-target-min-periods-3750/)），我們知道如何獲得更接近的匹配 - 在計算 $\ beta^a$ 滾動平均 $\langle .\rangle$ 應該完成整整 3750 分鐘。

由於這種變化
平均絕對誤差提高 0.00099 => 0.00006
最大絕對誤差提高 2.44 => 0.045

**從 2021 年 11 月 12 日起更新**

事實證明，我們一開始使用的樸素方法為 Recreated Target 提供了很多 NA 值，因此 Target 和 Recreated target 之間的比較僅針對部分數據進行（計算平均/最大誤差並不反映真實情況）。另外託管已發布的部分代碼，因此我們嘗試使用此代碼創建更快的當前計算版本。

平均絕對誤差 0.000190
最大絕對誤差 0.289100
標準差 0.001135

**2021 年 11 月 17 日更新**

* 在計算回報 R 時使用 x-1 而不是 log(x) - 請參閱 Branden Murray 的 [comment](https://www.kaggle.com/c/g-research-crypto-forecasting/discussion/286778#1582764)
* 使用唯一的時間戳（並非所有可能的分鐘都像以前的版本） - 檢查行“all_timestamps = np.sort(data['timestamp'].unique())”

最後一個更改意味著所有班次/滾動操作都不是以分鐘為單位，而是以可用記錄為單位。當所有資產都缺少時間戳時，就會出現差異。

平均絕對誤差 0.000190 => 0.000000

最大絕對誤差 0.289100 => 0.003798

標準偏差 0.001135 = 0.000001

# 幼稚的方法
在本節中，我們將根據 Target 計算描述進行逐步計算。 它很慢，並且在 Recreated Target 中給出了很多 NA 值。 因此，如果您需要更快/更正確的版本並且不了解它的計算方式，請跳轉到更優化部分。

讓我們從閱讀資產詳細信息、訓練數據和添加時間列開始。

In [ ]:
import os
import numpy as np
import pandas as pd
import gc

directory = '../input/g-research-crypto-forecasting'
file_path = os.path.join(directory, 'train.csv')
dtypes = {
    'timestamp': np.int64,
    'Asset_ID': np.int8,
#     'Count': np.int32,
#     'Open': np.float64,
#     'High': np.float64,
#     'Low': np.float64,
    'Close': np.float64,
#     'Volume': np.float64,
#     'VWAP': np.float64,
    'Target': np.float64,
}
data = pd.read_csv(file_path, dtype=dtypes, usecols=list(dtypes.keys()))
data['Time'] = pd.to_datetime(data['timestamp'], unit='s')

file_path = os.path.join(directory, 'asset_details.csv')
details = pd.read_csv(file_path)

然後按照下面的公式計算回報。

$$R^a(t) = log (P^a(t+16)\ /\ P^a(t+1))$$

這是分別為每個資產完成的。 我們不知道應該使用哪個價格。 有五種不同的價格：開盤價、最高價、最低價、收盤價和 VWAP。 可能有一個組合，比如時間 + 1 分鐘的開盤價和時間 + 16 分鐘的收盤價。 我們在下面的計算中使用**收盤**價格。

In [ ]:
price_column = 'Close'
ids = list(details.Asset_ID)
chunks = []
for id in ids:    
    asset = data[data.Asset_ID == id].copy()
    asset.sort_values(by='Time', inplace=True)
    asset.set_index(keys='Time', inplace=True)
    asset['p1'] = asset[price_column].shift(freq='-1T')
    asset['p16'] = asset[price_column].shift(freq='-16T')
    asset['r'] = np.log(asset.p16/asset.p1)
    asset.drop(['p1', 'p16'], axis=1, inplace=True)
    asset.reset_index(inplace=True)
    chunks.append(asset)

data = pd.concat(chunks)
data.sort_values(by='Time', inplace=True)

接下來，為每一行分配權重。 併計算 M(t)。 請注意，所有資產的 M(t) 都是相同的，並且僅取決於時間。
$$M(t) = \frac{\sum_a w^a R^a(t)}{\sum_a w^a}$$

我們不知道是應該為所有資產計算 ${\sum_a w^a}$ 還是只為在時間 t 有數據的資產計算 ${\sum_a w^a}$。


In [ ]:
data['w'] = data['Asset_ID'].map(details.set_index(keys='Asset_ID')['Weight'])
weight_sum = details.Weight.sum()

data['weighted_asset_r'] = data.w * data.r
time_group = data.groupby('Time')

m = time_group['weighted_asset_r'].sum() / time_group['w'].sum()
#m = time_group['weighted_asset_r'].sum() / weight_sum

data.set_index(keys=['Time'], inplace=True)
data['m'] = m
data.reset_index(inplace=True)

之後，計算 Beta。 括號 $\langle .\rangle$ 表示隨時間推移的滾動平均值（3750 分鐘窗口）。 如果沒有完整的 3750 分鐘窗口，$\beta$ 變為零。

$$\beta^a = \frac{\langle M \cdot R^a \rangle}{\langle M^2 \rangle}$$

In [ ]:
data['m2'] = data.m ** 2
data['mr'] = data.r * data.m

chunks = []
for id in ids:
    # type: pd.DataFrame
    asset = data[data.Asset_ID == id].copy()
    asset.sort_values(by='Time', inplace=True)
    asset.set_index(keys='Time', inplace=True)
    asset['mr_rolling'] = asset['mr'].rolling(window='3750T', min_periods=3750).mean()
    asset['m2_rolling'] = asset['m2'].rolling(window='3750T', min_periods=3750).mean()
    asset.reset_index(inplace=True)
    chunks.append(asset)
    debug = 1

data = pd.concat(chunks)
data.sort_values(by='Time', inplace=True)
data['beta'] = data['mr_rolling'] / data['m2_rolling']

最後計算目標。
$$\text{Target}^a(t) = R^a(t) - \beta^a M(t)$$

In [ ]:
data['Target_recreated'] = data['r'] - data['beta'] * data['m']

現在我們比較給定和重新創建的目標。

In [ ]:
data['Target_diff'] = np.abs(data['Target'] - data['Target_recreated'])

print(f'Average absolute error {data.Target_diff.mean():8.6f}')
print(f'Max absolute error {data.Target_diff.max():8.6f}')
print(f'Standard deviation {data.Target_diff.std():8.6f}')

正常目標在 [-0.5, 0.96] 範圍內更改，因此最大絕對誤差 2.4 意味著重新創建的目標對於某些記錄是完全錯誤的。

In [ ]:
data['Target'].agg(['min', 'max'])

讓我們檢查一下我們有多少重新創建的 Target 錯誤（值超出範圍）的記錄。

In [ ]:
(data.Target_recreated < -0.509351).sum()

In [ ]:
(data.Target_recreated > 0.96417).sum()

我嘗試了不同的價格，更改了公式中的時間間隔，將滾動平均值從 3750 分鐘替換為 3750 條最後記錄等，但結果相同或更糟。 有可能最小化最大誤差，但平均值變得更高。

如果您在計算中發現錯誤或對原始 Target 的實際計算方式有所了解，請告訴我。

事實證明，這段代碼在 Recreated Target 中給出了很多 NA 值，因此最小化 Abs/Max 誤差可能會產生誤導。

In [ ]:
pd.isna(data.Target).sum()

In [ ]:
pd.isna(data.Target_recreated).sum()

Resulting Recreated Target 具有超過 10 倍的 NA 值。 讓我們檢查一下這些 NA 是什麼時候引入的。

In [ ]:
pd.isna(data.r).sum()

在計算 R 時，NA 的數量幾乎等於目標 NA 的數量。差值是 1086。可以用每個資產每個月的 [missing first minite](https://www.kaggle.com/c/g-research-crypto-forecasting/discussion/286095) 來解釋。

有趣的是，目標存在於火車數據集中，日期為 YYYY-MM-01 23:44:00 (t+16) 和 YYYY-MM-01 23:59:00 (t+1)。對於這些時間，計算算法需要 YYYY-MM-01 00:00:00 時間的價格，但測試數據中沒有這個時間。它可以被認為是Target是在其他數據上計算的證明，然後用已經計算的Target構建訓練數據集。

這個缺少一個月的第一分鐘的錯誤給出了大約 44 個月 x 2（每一行影響輪班 1 和 16 的兩個目標）x 14（資產數量）=> 1232。一些資產沒有所有月份的數據。

唯一可以產生NA的地方是下面的代碼
> 滾動（窗口='3750T'，min_periods=3750）

3750 分鐘間隔內的缺失值會導致 NA 值。

現在讓我們看一下host提供的[code](https://www.kaggle.com/c/g-research-crypto-forecasting/discussion/286778)：
> num = df.multiply(mkt.values, axis=0).rolling(window).mean().values
> denom = mkt.multiply(mkt.values, axis=0).rolling(window).mean().values

滾動接受兩種可能的窗口類型 - [可以是](https://pandas.pydata.org/pandas-docs/version/1.3.0/reference/api/pandas.DataFrame.rolling.html) shift/int（要移動的數字或行數）或時間偏移量（要移動的分鐘數），但沒有 **min_periods** 參數。這意味著對於 int shift 它根本不會產生 NA，對於時間偏移它可能僅在最後 3750 分鐘沒有數據時才產生 NA。

代碼的下一行計算 beta 並用零刪除所有生成的非數字值：
> beta = np.nan_to_num(num.T / denom, nan=0., posinf=0., neginf=0.)

這裡的結論是，添加 min_periods=3750 看似改進了我們的指標，實際上添加了許多 NA 值，有效地最小化了這些指標計算的數據。主機代碼顯示滾動平均值不會產生額外的 NA 值。

# 更優化的版本

在這裡，我們嘗試做兩件事：解決 NA 的問題並使這段代碼運行得更快，以及重用來自主機的代碼。
來自主機的代碼表明，數據應該以稍微不同的方式重新呈現，以避免昂貴的基於分鐘的輪班/滾動操作，並立即對所有資產執行計算。

我們首先創建所有時間戳（分鐘）都存在的新數據框。 在火車上，某些資產可能會缺少幾分鐘。We start with creating new data frame where all timestamps (minutes) present. In the train some minutes may be missing for some assets.

In [ ]:
ids = list(details.Asset_ID)
asset_names = list(details.Asset_Name)

# times = data['timestamp'].agg(['min', 'max']).to_dict()
# all_timestamps = np.arange(times['min'], times['max'] + 60, 60)
all_timestamps = np.sort(data['timestamp'].unique())
targets = pd.DataFrame(index=all_timestamps)


接下來，我們計算每個資產的 R 並將其值作為列添加到新的目標數據框。 請注意，某些行將包含 NA，因為可能缺少帶班次的所需價格。

In [ ]:
for i, id in enumerate(ids):
    asset = data[data.Asset_ID == id].set_index(keys='timestamp')
    price = pd.Series(index=all_timestamps, data=asset[price_column])
#     targets[asset_names[i]] = np.log(
#         price.shift(periods=-16) /
#         price.shift(periods=-1)
#     )
    targets[asset_names[i]] = (
        price.shift(periods=-16) /
        price.shift(periods=-1)
    ) - 1



接下來計算 M 作為每行的列的加權平均值（當前為每個資產保存 R 值）。 此實現將不可用的 R 值計數為零。 請注意，在計算加權平均值時，這不是處理 NA 值的唯一方法。

In [ ]:
weights = np.array(list(details.Weight))
targets['m'] = np.average(targets.fillna(0), axis=1, weights=weights)

最後是時候應用主機提供的代碼來計算 beta 和 Target。

In [ ]:
m = targets['m']

num = targets.multiply(m.values, axis=0).rolling(3750).mean().values
denom = m.multiply(m.values, axis=0).rolling(3750).mean().values
beta = np.nan_to_num(num.T / denom, nan=0., posinf=0., neginf=0.)

targets = targets - (beta * m.values).T

讓我們檢查一下它與給定的 Target 有何不同以及我們有多少 NA 值。

In [ ]:
diffs = []

for i, id in enumerate(ids):
    print(asset_names[i])
    # type: pd.DataFrame
    asset = data[data.Asset_ID == id].set_index(keys='timestamp')
    print(f'asset size {asset.shape[0]}')
    recreated = pd.Series(index=asset.index, data=targets[asset_names[i]])
    diff = np.abs(asset['Target'] - recreated)
    diffs.append(diff[~pd.isna(diff)].values)
    print(f'Average absolute error {diff.mean():8.6f}')
    print(f'Max absolute error {diff.max():8.6f}')
    print(f'Standard deviation {diff.std():8.6f}')
    print(f'Target na {pd.isna(asset.Target).sum()}')
    print(f'Target_calculated na {pd.isna(recreated).sum()}')
    print()

diffs = np.concatenate(diffs, axis=0)
print('For all assets')
print(f'Average absolute error {diffs.mean():8.6f}')
print(f'Max absolute error {diffs.max():8.6f}')
print(f'Standard deviation {diffs.std():8.6f}')


它仍然不理想，但至少我們的指標是根據幾乎所有可用數據計算得出的，NA 的數量與 Target 中的幾乎相同。


有一個關於 Beta 等於零的討論。 它將目標計算僅轉換為 R 計算。 讓我們看看有多少非數字值以及有多少行 beta=0。

In [ ]:
beta_ = num.T / denom

for i, id in enumerate(ids):
    print(asset_names[i])
    print(f'Infiinte beta rows {np.isinf(beta_[i]).sum()}')
    nan_sum = np.isnan(beta_[i]).sum()
    print(f'NAN beta rows {nan_sum} ({100 * nan_sum / beta_.shape[1]:5.2f}%)')

    eps = 1e-6
    zero_sum = ((beta[i] > -eps) & (beta[i] < eps)).sum()
    print(f'Zero beta rows {zero_sum} ({100 * zero_sum / beta.shape[1]:5.2f}%)')
    print()

Below is the same code but in the form of a function you can copy and paste. Note that it does not require Time column, it uses timestamp instead.

In [ ]:
def calculate_target(data: pd.DataFrame, details: pd.DataFrame, price_column: str):
    ids = list(details.Asset_ID)
    asset_names = list(details.Asset_Name)
    weights = np.array(list(details.Weight))

    all_timestamps = np.sort(data['timestamp'].unique())
    targets = pd.DataFrame(index=all_timestamps)

    for i, id in enumerate(ids):
        asset = data[data.Asset_ID == id].set_index(keys='timestamp')
        price = pd.Series(index=all_timestamps, data=asset[price_column])
        targets[asset_names[i]] = (
            price.shift(periods=-16) /
            price.shift(periods=-1)
        ) - 1
    
    targets['m'] = np.average(targets.fillna(0), axis=1, weights=weights)
    
    m = targets['m']

    num = targets.multiply(m.values, axis=0).rolling(3750).mean().values
    denom = m.multiply(m.values, axis=0).rolling(3750).mean().values
    beta = np.nan_to_num(num.T / denom, nan=0., posinf=0., neginf=0.)

    targets = targets - (beta * m.values).T
    targets.drop('m', axis=1, inplace=True)
    
    return targets